## HOMEWORK: $Z$ boson production in $p+\bar{p}$ collisions
### Generate 10000 $p+\bar{p}$ events at 630 GeV of $Z$ boson production ($Z$ discovery at CERN SPS). (in the center of mass)

Setup:
* in order to look at $Z$ boson production we limit physics list of Pythia8 to include only hard $f\bar{f}\to Z \to f\bar{f}$ processes
  * `WeakSingleBoson:ffbar2gmZ`
* moreover, we will disable all $Z$ decay modes other than $Z\to e^+e^-$, in which the particle was discovered
  * `id:onMode = off` disables all decay channels of particle with given id (`id` should be an integer equal to PDG code of the particle)
  * `id:onIfAny = prod` enables decay of particle `id` in channels containing `prod` - see pythia documentation if more info is necessary
* to speed up the simulation we disable hadronisation (we are interested only in leptonic final states)
* we also limit phase space to $\pm 20$ GeV around the $Z$ rest mass
  * use `PhaseSpace:mHatMin = XX` and `PhaseSpace:mHatMax = XX` options, values are in GeV

To do:
* select candidates for products of $Z$ decay in each event and plot their invariant mass distribution in range 70-110 GeV
* check influence of final state radiation on the spectrum - generate additional 10000 events with FSR turned off
* normalize histograms so that their maxima are at $1$
* draw both distributions (as a histograms, option `"hist"`) on the same plot with axis labels and a legend, how does FSR influence invariant mass spectrum?
* draw $\frac{1}{p_T}\frac{\mathrm{d}N}{\mathrm{d}p_T}$ and $\frac{\mathrm{d}N}{\mathrm{d}\eta}$ spectra of produced $Z$ bosons (only for one dataset, with FSR on), with axis labels and proper normalisation, pick the $x$ axis ranges so that the whole distribution fits inside the plot
  * **hint:** $Z$ boson will occur multiple times in the event listing (why?), it will never be a final particle, pick only $Z$'s that appear *last* in each event
* what is the *mean* number $\langle N_Z\rangle$ of $Z$ bosons produced in an event, with what error? is this result surprising?
  * **hint:** $\langle N \rangle = \int\limits_0^\infty \frac{\mathrm{d}N}{\mathrm{d}p_T}\cdot \mathrm{d}p_T = \int\limits_{-\infty}^\infty \frac{\mathrm{d}N}{\mathrm{d}\eta}\cdot \mathrm{d}\eta$, see `TH1::Integral`, and `TH1::IntegralAndError` methods and make sure, that *"width"* option is used so that bin contents are multiplied by bin widths during computation of the integral

### Root and necessary packages

In [16]:
import ROOT
import os, sys, subprocess, time
import py8settings as py8s
from py8settings import PDF
import itertools as it
import ctypes

%jsroot on

#### Compile pythia code

In [17]:
!cd pythia-examples; make main300; cd -

make: 'main300' is up to date.
/scratch/home/asus/MPPE/a/Modern_Particle_Physics_Experiments/Homeworks


### Pythia8 view settings and view events; PARAMETERS FOR INCOMING BEAMS

In [18]:
cmnd_file = "settings.cmnd"

#we start fresh
if os.path.exists(cmnd_file):
    os.remove(cmnd_file)
os.system("touch "+cmnd_file)

#beam settings:
py8s.beam_settings(cmnd_file)

# beam energies are in Gev

You can now set the parameters for the incoming beams:


interactive(children=(Dropdown(description='beam A id  [Beams:idA]', index=9, layout=Layout(width='750px'), op…

### Select the number of events, settings for the random seed, and processes

In [19]:
py8s.basic_settings(cmnd_file)

# we can choose what bosons are in our final state

You can now select the number of events, settings for the random seed, and processes:


interactive(children=(IntText(value=100, description='# of events  [Main:numberOfEvents]', layout=Layout(width…

In [20]:
py8s.onoff_settings(cmnd_file)

You can switch on/off different parts of the simulation:


interactive(children=(Checkbox(value=True, description='multi-parton interactions  [PartonLevel:MPI]', layout=…

In [21]:
# Write settings for phase-space cuts to file.
py8s.pscuts_settings(cmnd_file, 2)

You can now select the phase-space cuts for 2 final-state particles:


interactive(children=(FloatText(value=4.0, description='minimum invariant mass in GeV  [PhaseSpace:mHatMin]', …

In [23]:
# Write additional settings if needed.
py8s.more_settings(cmnd_file, ["23:onMode = off", "23:onIfAny = 11 -11"])

In [24]:
# Start pythia+dire.
command = "./pythia-examples/main300 --visualize_event --input "+cmnd_file  #+ ">> /dev/null"
subprocess.call(["bash","-c",command]);


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.306      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 28 Jun 2021  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 21 Mar 2022 at 16:17:05    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

### Our pythia configuration

In [15]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    pythia.ReadString("23:onMode = off")    # disable decay of Z boson
    pythia.ReadString("23:onIfAny = 11 -11")    # enable decay of Z boson in channels containing electron and positon
    

pythia = ROOT.TPythia8(False)   # suppress the banner message
makePythiaSilent(pythia)
pythia.ReadString("HardQCD:all = on")   # passing set of parameters, options in Pythia documentation; 
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42") # to ro randomize seed; while we give the same seed we will receive the same results
pythia.Initialize(2212 , -2212 , 14000)  # 2212 - proton in PDG codes, antiproton has a minus sign
particles = ROOT.TClonesArray("TParticle", 1000) #prepare space for generated particles

# Warning about cross section violated is ok

 PYTHIA Warning in PhaseSpace2to2tauyz::trialKin: maximum for cross section violated  


### My solution

In [ ]:
nEvents=10000
pythia = ROOT.TPythia8(False)                       # false to suppress banner
makePythiaSilent(pythia)                            # suppress most of pythia console output
#pythia setup and initialization goes here
particles = ROOT.TClonesArray("TParticle", 1000)    # prepare space for generated particles

#your solution goes here